In [1]:
from cloudvolume import CloudVolume, view
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cc3d
from tifffile.tifffile import imwrite
%pwd

For now, I'll only look at a tiny region of entire dataset. So we need to think about how to scale up this approach later.

In [2]:
image_cv = CloudVolume(
    'precomputed://gs://lee-lab_female-adult-nerve-cord_tokyo-storage/alignmentV4/em/rechunked',
    progress=False,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True
)

nuclei_cv = CloudVolume(
    'precomputed://gs://ranl_scratch_zetta_30/neuroglancer/nuclei/3d528bc3def9f51e09ae55e68ee46518',
    progress=True,
    cache=True, # cache to disk to avoid repeated downloads
    use_https=True
)

In [3]:
image = image_cv.download_point( (52785, 86885, 1894), mip=4, size=(128, 128, 256) ) # mip2+ only
# mip needs to be the same?
# 4.3*2^4*128/45=196
# 30s

In [4]:
nuclei = nuclei_cv.download_point( (52785, 86885, 1894), mip=[68.8,68.8,45.0], size=(128, 128, 256) ) # mip 0 or 4 only
# 10min!?

Downloading: 100%|██████████| 162/162 [00:02<00:00,  7.22it/s]

In [ ]:
# view(image)

In [ ]:
# view(nuclei)

In [5]:
test = image[:,:,:]
testn_temp = nuclei[:,:,:]
testn = np.where(testn_temp > 0.5, 1, 0)
type(testn)

numpy.ndarray

In [6]:
view(nuclei)
# 9 cell bodies

# plt.imshow(testn_temp[:,:,149])
# plt.imshow(testn[:,:,149])

Viewer server listening to http://localhost:8080

Continuing program execution...


In [7]:
# print(testn.ndim) 
# 4
# print(testn.size) 
# print(testn.shape) 
# (128, 128, 256, 1)
testn_s = np.squeeze(testn)
print(testn_s.shape) 

(128, 128, 256)


In [8]:
# 26-connectivity
# https://en.wikipedia.org/wiki/Pixel_connectivity
# https://github.com/seung-lab/connected-components-3d
labels_in = testn_s
labels_out, N = cc3d.connected_components(labels_in, return_N=True, connectivity=26) # free
print(N)
# 15

15


It has to be 9, but 15...

In [15]:
# write numpy array into tiff using tifffile
# pip install tifffile
volume = testn_s.astype('float32')
imwrite('../Output/testn_s.tif', volume, imagej=True)

In [17]:
for segid in range(1, N+1):
  extracted_image = labels_out * (labels_out == segid)

In [19]:
print(labels_out.shape)

(128, 128, 256)


In [20]:
labels_out

array([[[ 0,  0,  0, ...,  8,  8,  8],
        [ 0,  0,  0, ...,  0,  0,  8],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  8,  8,  8],
        [ 0,  0,  0, ...,  8,  8,  8],
        [ 0,  0,  0, ...,  0,  8,  8],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  8,  8,  8],
        [ 0,  0,  0, ...,  8,  8,  8],
        [ 0,  0,  0, ...,  8,  8,  8],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 1,  1,  1, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  0,  0,  0],
        [ 1,  1,  1, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ..., 12, 12, 12],
        [ 0,  0,  0, ..., 12, 12, 12],
        [ 0,  0,  0, ..., 12, 12

In [ ]:
image_cv.cache.flush()
nuclei_cv.cache.flush() 

In [ ]:
xmax-xmin
ymax-ymin
zmax-zmin